# Librerias y carga de datos

In [ ]:
import numpy as np
import pandas as pd
from getpass import getpass
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm



df = pd.read_csv("/content/df.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   video_key         1000 non-null   object 
 1   transcription     1000 non-null   object 
 2   is_song           1000 non-null   object 
 3   duration_sec      1000 non-null   float64
 4   descripcion       1000 non-null   object 
 5   author_id         1000 non-null   object 
 6   followers         1000 non-null   int64  
 7   following         1000 non-null   int64  
 8   account_creation  1000 non-null   object 
 9   video_creation    1000 non-null   object 
 10  likes             1000 non-null   int64  
 11  comments          1000 non-null   int64  
 12  shares            1000 non-null   int64  
 13  n_hashtags_used   1000 non-null   int64  
 14  hashtags_used     908 non-null    object 
 15  engagement_rate   1000 non-null   float64
 16  viral             1000 non-null   int64  
d

# Limpieza simple de los datos

In [ ]:
df['is_song'] = df['is_song'].replace({'No':'no'})
df['is_song'] = df['is_song'].replace({'Yes':'yes'})

In [ ]:
df['is_song'] = df['is_song'].replace({'yes': 1, 'no': 0})




# Verificar el resultado
print(df['is_song'].value_counts())

is_song
1    614
0    386
Name: count, dtype: int64


/tmp/ipython-input-2258975603.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_song'] = df['is_song'].replace({'yes': 1, 'no': 0})


In [ ]:

df["hashtags_used"] = (
    df["hashtags_used"]
    .fillna("")  # evitar NaN
    .str.replace(r"\s+", "", regex=True)  # quitar espacios extra
    .apply(lambda x: " ".join([f"#{tag}" for tag in x.split(",") if tag != ""]))
)


In [ ]:
df

,video_key,transcription,is_song,duration_sec,descripcion,author_id,followers,following,account_creation,video_creation,likes,comments,shares,n_hashtags_used,hashtags_used,engagement_rate,viral
0,6886060490687106306.mp4,yo!'.,1,13.88,se ve a un hombre con una barba y una camiset...,user_00099,22220,8341,11/27/23,5/12/25,891,115,64,3,#dance #life #gaming,4.815482,1
1,6886115520425266433.mp4,welkom bij de eerste aflevering van jan-pieter...,0,33.17,vemos a un hombre sentado en una silla frente...,user_00091,100083,5005,10/13/24,10/15/25,2005,167,36,2,#music #fyp,2.206169,0
2,6877179236386376961.mp4,kijk eens naar de video's van de serie 'kantja...,0,59.03,se muestra una persona que está preparando un...,user_00147,26324,3742,12/1/20,2/1/25,157,18,7,0,,0.691384,0
3,6886527878100438273.mp4,i'm on some dumb shit i got away what he say h...,1,46.28,se ve a una persona dibujando en una pantalla...,user_00151,15425,9121,3/6/25,5/2/25,329,30,20,0,,2.457050,0
4,6883894825855274242.mp4,the'.,0,11.80,se muestra una escena de un juego de videojue...,user_00162,468764,7385,11/20/22,12/16/22,9717,734,226,1,#dance,2.277692,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,6891747931905330433.mp4,just alone as you swear yeah shit for me so da...,1,15.07,se ve a una joven rubia con cabello largo y o...,user_00060,625,8965,8/27/22,11/22/24,16,2,0,1,#travel,2.880000,0
996,6878231837710372098.mp4,foreign'.,1,10.20,se muestra una persona montada en un biciclet...,user_00154,4536,1241,6/27/19,8/20/19,125,17,3,5,#fyp #gaming #dance #music #travel,3.196649,0
997,6907635850985704709.mp4,the'.,1,32.87,se muestra a un hombre musculoso en un entorn...,user_00026,7079,7110,8/16/21,9/1/23,249,17,11,1,#food,3.912982,0
998,6904743507693554950.mp4,bitch i'm on a rampage mask on my damn face'.,1,17.77,"se muestra un hombre en un gimnasio, trabajan...",user_00068,48584,7979,8/5/19,10/20/20,956,73,15,0,,2.148856,0


# Modelo facebook/nllb-200-distilled-1.3B

In [ ]:
# Cargar modelo NLLB
model_id = "facebook/nllb-200-distilled-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")

# Configurar idiomas
src_lang = "spa_Latn"  # Español latinoamericano
tgt_lang = "eng_Latn"  # Inglés

# Traducir columna "descripcion"
translated = []
for text in tqdm(df["descripcion"].fillna("").tolist()):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to("cuda")
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang))
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    translated.append(translated_text)

df["descripcion_en"] = translated


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]


100%|██████████| 1000/1000 [20:06<00:00,  1.21s/it]


# Descargar archivo CSV

In [ ]:
df.to_csv("/content/df_traducido.csv", index=False)
from google.colab import files
files.download("/content/df_traducido.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>